# 不知道叫啥

In [1]:
# 导入所有需要的包
import math
import numpy as np
import pandas as pd
import geopandas as gpd
from shapely.geometry import Polygon, LineString
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns

## 1 数据导入

In [3]:
# 读取上海市2018年9月1日的共享单车数据
data = pd.read_csv(r'./assets/data/data0901.csv')
data

,BIKE_ID,DATA_TIME,LOCK_STATUS,LONGITUDE,LATITUDE
0,713ED7B044393233E0533C0BA8C09291,2018-09-01 0:00:49,0,121.531489,31.263296
1,713ED78FC0813233E0533C0BA8C09291,2018-09-01 0:00:49,0,121.363938,31.212551
2,7169493311DD0D74E0533E0BA8C01BB4,2018-09-01 0:00:12,1,121.488291,31.264857
3,713ED795F1033233E0533C0BA8C09291,2018-09-01 0:00:06,1,121.345189,31.135745
4,713ED7AF8E213233E0533C0BA8C09291,2018-09-01 0:00:49,0,121.364748,31.258107
...,...,...,...,...,...
3063365,713ED79FD6C23233E0533C0BA8C09291,2018-09-01 10:25:50,1,121.449317,31.246356
3063366,713ED79F3BAF3233E0533C0BA8C09291,2018-09-01 14:46:43,1,121.429002,31.183046
3063367,713ED7969D603233E0533C0BA8C09291,2018-09-01 2:49:39,1,121.223636,31.231712
3063368,713ED7A760F03233E0533C0BA8C09291,2018-09-01 18:24:33,1,121.673009,31.301073


此时，数据中的 `DATA_TIME` 字段中的时间为字符串格式，直接将其格式转换为 `pandas` 内置的时间格式以便后续排序，并粗略筛选研究范围内的数据。

In [4]:
# 转换数据的时间
data['DATA_TIME'] = pd.to_datetime(data['DATA_TIME'])
# 筛选研究范围内的数据
data = data[(data['LONGITUDE']<122)&
            (data['LONGITUDE']>120)&
            (data['LATITUDE']<32)&
            (data['LATITUDE']>30)]

取出其中一辆单车的数据，并按时间顺序对数据排序，以便观察数据的采集情况。可以看到，单独取出一辆单车的数据并以时间排序，数据的锁状态为 `0` 和 `1` 交替出现。

In [5]:
tmp = data[data['BIKE_ID']=='713ED79C59CC3233E0533C0BA8C09291']
tmp.sort_values(by = ['DATA_TIME'])

,BIKE_ID,DATA_TIME,LOCK_STATUS,LONGITUDE,LATITUDE
1767607,713ED79C59CC3233E0533C0BA8C09291,2018-09-01 14:06:54,0,121.550754,31.247030
1823717,713ED79C59CC3233E0533C0BA8C09291,2018-09-01 14:15:11,1,121.538810,31.240057
2278567,713ED79C59CC3233E0533C0BA8C09291,2018-09-01 16:45:28,0,121.538906,31.239970
2249403,713ED79C59CC3233E0533C0BA8C09291,2018-09-01 16:55:58,1,121.550667,31.247302
2370822,713ED79C59CC3233E0533C0BA8C09291,2018-09-01 17:50:09,0,121.550707,31.247015
2483733,713ED79C59CC3233E0533C0BA8C09291,2018-09-01 17:58:10,1,121.548652,31.241853
2545391,713ED79C59CC3233E0533C0BA8C09291,2018-09-01 18:43:44,0,121.548426,31.241471
2529933,713ED79C59CC3233E0533C0BA8C09291,2018-09-01 18:54:49,1,121.540243,31.248092


## 2 出行链重构